## OJS Submission Export

Code for generating a CSV file for all submissions during a given date range from a local version of the database. Runs on Python3. Additional authors are printed within "{}".

In [1]:
import pymysql.cursors

In [2]:
# Connect to the database
db = pymysql.connect(host='localhost',
                            user='root',
                            password='',
                            db='ojs2',
                            cursorclass=pymysql.cursors.DictCursor)

In [3]:
journal_id = 10

In [4]:
# For submission: ID | Title | Author | Abstract (if possible) |  Section (if possible)

In [5]:
import time   
start_date = time.strftime('2016-02-15 00:00:00')
end_date = time.strftime('2016-03-22 13:00:00')

In [6]:
# Article ID, Article Author, Article Title, Section ID

with db.cursor() as cursor:
    sql = ("""SELECT articles.article_id, article_settings.setting_value, articles.section_id 
        FROM articles 
        LEFT JOIN (article_settings) 
            on (articles.article_id = article_settings.article_id)
        WHERE articles.journal_id = %s 
            AND articles.date_submitted BETWEEN %s and %s 
            AND article_settings.setting_name LIKE 'cleanTitle'""")
    cursor.execute(sql, (journal_id, start_date, end_date))
    result = cursor.fetchall()

In [7]:
result

[{'article_id': 1359,
  'section_id': 47,
  'setting_value': 'From the cockpit to the classroom: leveraging failure as a reflective learning opportunity'},
 {'article_id': 1360,
  'section_id': 29,
  'setting_value': 'test of submission system'},
 {'article_id': 1370,
  'section_id': 29,
  'setting_value': 'Teaching scientific communication using infographics'},
 {'article_id': 1365,
  'section_id': 29,
  'setting_value': 'MLearning Mobile Learning on LinkedIn:  Interactive Online Learning Strategies for Increasing Studentsâ\x80\x99 Functional and Industry Intelligence'},
 {'article_id': 1367,
  'section_id': 46,
  'setting_value': 'Diagramming Prior Knowledge in the Classroom'},
 {'article_id': 1369,
  'section_id': 47,
  'setting_value': 'â\x80\x9cTranslationalâ\x80\x9d Research in Motion: Some Suggested Approaches'},
 {'article_id': 1371,
  'section_id': 47,
  'setting_value': 'Developing a scaffolded learning process to increase undergraduate research skills'},
 {'article_id': 1372

In [8]:
for article in result:
    article_id = article.get('article_id')
    section_id = article.get('section_id')
    with db.cursor() as cursor:
        sql1 = """SELECT article_settings.setting_value
        FROM article_settings
        WHERE article_settings.article_id = %s
        AND setting_name LIKE 'abstract'"""
        
        sql2 = """SELECT section_settings.setting_value
        FROM section_settings
        WHERE section_settings.section_id = %s
        AND setting_name LIKE 'title'"""
        
        sql3 = """ SELECT authors.first_name, authors.last_name
        FROM authors
        WHERE authors.submission_id = %s"""
        
        cursor.execute(sql1, (article_id))
        abstract_dict = cursor.fetchone()
#         print(abstract_dict)
        abstract = abstract_dict.get('setting_value')
#         print(abstract)
        cursor.execute(sql2, (section_id))
        section_dict = cursor.fetchone()
        section = section_dict.get('setting_value')
        
        cursor.execute(sql3, (article_id))
        authors_dict = cursor.fetchall()
#         print(authors_dict)
        
        author1 = authors_dict[0]
        firstname = author1['first_name']
        lastname = author1['last_name']
        try:
            additional_authors = authors_dict[1:]
        except:
            pass

        article.update({'abstract': abstract, 'section': section, 'firstname': firstname, 'lastname': lastname, 'additional_authors': additional_authors })
        
    

In [9]:
result

[{'abstract': '<p>Marine Corps fighter pilots and multilingual and first-generation college students share several commonalities as learners: they have overcome extraordinary challenges to earn hard-fought positions within their respective communities; they work within syllabus-driven, high-stakes learning environments; and they face immense pressure to succeed, yet they also need to be well-equipped to cope with failure. In this session, presenters will introduce <em>lessons learned</em> as an activity that supports failure as a valuable component in the reflective learning process Â\xa0(Cannon, &amp; Edmondson, 2005; Kapur, 2008) in two very different, yet complementary, learning communities.</p><p>Lessons learned is a pedagogical tool employed in Marine Corps fighter squadrons in which a group of highly trained, competitive fighter pilots are required to self-identify, critically examine, and discuss their recent failures in the aircraft. In a community where 80% of mishaps are at l

In [10]:
import csv

In [11]:
keys = result[0].keys()
with open('ITLCP-Submission-2016.csv', 'w') as f:  # Just use 'w' mode in 3.x
    dict_writer = csv.DictWriter(f, keys)
    dict_writer.writeheader()
    dict_writer.writerows(result)